# Dataset import and exploration
- https://www.kaggle.com/shelvigarg/wine-quality-dataset

In [ ]:
import numpy as np
import pandas as pd 

df = pd.read_csv('data/winequalityN.csv')
df.sample(5)

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

### Drop missing values:

In [ ]:
df = df.dropna()
df.isnull().sum()

### Encode string data:

In [ ]:
df['type'].value_counts()

In [ ]:
df['is_white_wine'] = [1 if typ == 'white' else 0 for typ in df['type']]
df.drop('type', axis=1, inplace=True)

df.head()

### All data is numeric now:

In [ ]:
df.dtypes

<br>

# Convert to a binary classification problem
- This is not a binary classification problem by default
- We can make it one by declaring wines above some quality point good wines and rest of them bad wines

In [ ]:
df['quality'].value_counts()

- So we'll have 63.3% good wines and the rest are bad

In [ ]:
len(df[df['quality'] >= 6]) / len(df)

In [ ]:
df['is_good_wine'] = [1 if quality >= 6 else 0 for quality in df['quality']]
df.drop('quality', axis=1, inplace=True)

df.head()

In [ ]:
df['is_good_wine'].value_counts()

In [ ]:
df.head()

<br>

# Train/Test split

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('is_good_wine', axis=1)
y = df['is_good_wine']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, random_state=42
)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
X_train.head()

<br>

# Data scaling
- Input features aren't on the same scale, so we'll fix it quickly:

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
X_train_scaled[:3]

<br>

# Model training

In [ ]:
import tensorflow as tf

- This is a completely random neural network architecture
- Use `sigmoid` as the activation function in the last layer when working with binary classification problems
- Use `binary_crossentropy` as a loss function when working with binary classification problems
- We'll track accuracy, precision, and recall and train for 100 epochs

In [ ]:
tf.random.set_seed(42)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    loss=tf.keras.losses.binary_crossentropy,
    optimizer=tf.keras.optimizers.Adam(lr=0.03),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

history = model.fit(X_train_scaled, y_train, epochs=100)

<br>

# Model performance visualization

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams

rcParams['figure.figsize'] = (18, 8)
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False

In [ ]:
plt.plot(np.arange(1, 101), history.history['loss'], label='Loss')
plt.plot(np.arange(1, 101), history.history['accuracy'], label='Accuracy')
plt.plot(np.arange(1, 101), history.history['precision'], label='Precision')
plt.plot(np.arange(1, 101), history.history['recall'], label='Recall')
plt.title('Evaluation metrics', size=20)
plt.xlabel('Epoch', size=14)
plt.legend();

- You could keep training the model, as accuracy, precision, and recall seem to grow slightly

<br>

# Making predictions

In [ ]:
predictions = model.predict(X_test_scaled)
predictions

- These are probabilities - here's how to convert them to classes (threshold = 0.5)

In [ ]:
prediction_classes = [1 if prob > 0.5 else 0 for prob in np.ravel(predictions)]
print(prediction_classes[:20])

<br>

# Model evaluation

- Evaluation on the test set:

In [ ]:
loss, accuracy, precision, recall = model.evaluate(X_test_scaled, y_test)
loss, accuracy, precision, recall

In [ ]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, prediction_classes))

- 383 True Negatives, 597 True positives, 214 False negatives, 99 False positives
- Further evaluation:

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

print(f'Accuracy:  {accuracy_score(y_test, prediction_classes):.2f}')
print(f'Precision: {precision_score(y_test, prediction_classes):.2f}')
print(f'Recall:    {recall_score(y_test, prediction_classes):.2f}')